In [4]:
pip install atproto

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.1/337.1 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.7/671.7 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 14.1
    Uninstalling websockets-14.1:
      Successfully uninstalled websockets-14.1
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [5]:
# From the transformers package, import ViTFeatureExtractor and ViTForImageClassification
from transformers import ViTFeatureExtractor, ViTForImageClassification

# From the PIL package, import Image and Markdown
from PIL import Image

# import requests
import requests

# import torch
import torch

# import matplotlib
import matplotlib.pyplot as plt

# url getter for mpl
import urllib

import numpy as np

# import bluesky api
from atproto import Client

In [18]:
# Load the feature extractor for the vision transformer
feature_extractor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224')
# Load the pre-trained weights from vision transformer
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

In [35]:
# 281: 'tabby, tabby cat'
# 282: 'tiger cat', 283: 'Persian cat', 284: 'Siamese cat, Siamese', 285: 'Egyptian cat', 286: 'cougar, puma, catamount, mountain lion, painter, panther, Felis concolor', 287: 'lynx, catamount', 288: 'leopard, Panthera pardus', 289: 'snow leopard, ounce, Panthera uncia', 290: 'jaguar, panther, Panthera onca, Felis onca', 291: 'lion, king of beasts, Panthera leo', 292: 'tiger, Panthera tigris', 293: 'cheetah, chetah, Acinonyx jubatus',
# 281 to 293
cat_labels = set()
for i in range(281, 294):
  cat_labels.add(i)

# these labels are to remove drawings, memes/reposts, and images with a lot of text respectively
bad_labels = {
917 : 'comic book', 916 : 'web site, website, internet site, site', 921 : 'book jacket, dust cover, dust jacket, dust wrapper'}

def test_bsky_image(url):

  f = urllib.request.urlopen(url)
  image = plt.imread(f, format='jpeg')
  # plt.imshow(image)
  inputs = feature_extractor(images=image, return_tensor="pt")
  pixel_values = inputs["pixel_values"]
  pixel_values = np.array(pixel_values)
  pixel_values = torch.tensor(pixel_values)
  outputs = model(pixel_values)
  logits = outputs.logits
  predicted_class_idx = logits.argmax(-1).item()
  sorted_preds = torch.argsort(logits, descending=True)[0]
  top_predictions = [sorted_preds[i].item() for i in range(50)] # 50 is semi-arbitrary based on our findings from testing pics
  top_values = [logits[0][pred].item() for pred in top_predictions]
  # print('label predictions', top_predictions)
  # print('values of predictions', top_values)
  found_cat_label = -1
  found_bad_label = -1
  bad_labels_found = []
  cat_score = 0
  for i, pred in enumerate(top_predictions):
    predicted_class = model.config.id2label[pred]
    # print(predicted_class)
    if pred in cat_labels:
      if found_cat_label == -1:
        found_cat_label = i
      cat_score += top_values[i]
    if pred in bad_labels:
      if found_bad_label == -1:
        found_bad_label = i
      bad_labels_found.append(pred)
      bad_labels_found.append(bad_labels[pred])
      cat_score -= top_values[i]
  # print(' ')
  print('    found cat label:', found_cat_label)
  print('    found bad label:', found_bad_label, bad_labels_found)
  would_pass = found_cat_label >= 0 and found_bad_label < 0
  # print('AI cat score: ', cat_score)
  print('    passed cat test', would_pass)
  return would_pass

In [7]:
client = Client()
bsky_username = ''
bsky_password = ''

client.login(bsky_username, bsky_password)

ProfileViewDetailed(did='did:plc:ktkc7jfakxzjpooj52ffc6ra', handle='tyrowo.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:ktkc7jfakxzjpooj52ffc6ra/bafkreiepqgg5tlozvv4bw5ficwwixflfprionhnli26hngc3hjn6ygpamu@jpeg', banner='https://cdn.bsky.app/img/banner/plain/did:plc:ktkc7jfakxzjpooj52ffc6ra/bafkreierasgybimbz3pfobqfmay5ufxm7jlqnjeiumdx7dyof6mcss7a4m@jpeg', created_at='2023-08-28T14:18:41.780Z', description='Self-taught professional developer, exF2P MTG Arena Challenger, ex wr holder speedrunning HotD2100%, ex streamer, ex melee player. \nGrilling, coding, magic, and posting pictures of my cat. \nNO POLITICS\nhe/him', display_name='Tyro, ft Ricky', followers_count=483, follows_count=1146, indexed_at='2025-01-14T22:58:49.714Z', joined_via_starter_pack=None, labels=[], pinned_post=Main(cid='bafyreihszjgxkh5fmu5je5eldv6i27ubndf

In [32]:
def like_post_and_add_user(post):
  user_did = post.author.did
  followed_user = client.follow(user_did).uri
  post_cid = post.cid
  post_uri = post.uri
  liked_post = client.like(uri=post_uri, cid=post_cid).uri
  print(f'      ✓✓✓ Successfully liked post and followed author.')
  # TODO - add this friend and the followed-user uri to the database

In [33]:
# input variables - x is target follows y is num of posts we want to look through, whichever end criteria we reach first
EMBEDDED_PIC = 'app.bsky.embed.images#view'

def follow_more_users(post_count, follows_count):
  data = client.app.bsky.feed.get_feed({
      'feed': 'at://did:plc:jfhpnnst6flqway4eaeqzj2a/app.bsky.feed.generator/cats',
      'limit': post_count,
  }, headers={})

  feed = data.feed
  next_page = data.cursor

  count = 0

  for i, f in enumerate(data.feed):
    you_follow_them = f.post.author.viewer.following
    you_are_followed_by = f.post.author.viewer.followed_by
    if not f.post.embed or f.post.embed.py_type != EMBEDDED_PIC:
      print(f'{i} ✗ no pic for post {i}')
    elif you_follow_them or you_are_followed_by:
      print(f'{i} ✗ {f.post.author.handle + " follows you" if you_are_followed_by else ""}{" and " if you_follow_them and you_are_followed_by else ""}{"you already follow " + f.post.author.handle if you_follow_them else ""}.')
    ## TODO - add elif check here for if they are in the database as someone who was once our friend? not necessary, could just immediately cycle them back into the friend rotation, but do this if you don't want to readd people
    else:
      print(i, '✓', f.post.embed.images[0].fullsize)
      is_cat = test_bsky_image(f.post.embed.images[0].fullsize)
      if is_cat:
        print(f'    ✓✓ successfully found cat pic at post {i}.')
        like_post_and_add_user(f.post)
        count += 1
        if count == follows_count:
          print(f'Successfully followed {count} new users!')
          return True
      else:
        print(f'    ✓✗ post {i} was not a cat pic')

  print(f'ran out of posts to check for new followers. followed {count} new users.')
  return False

  # print(data.feed[0].post.embed.images[0].fullsize)

In [34]:
follow_more_users(100, 25)

0 ✗ isabelsantos.bsky.social follows you and you already follow isabelsantos.bsky.social.
1 ✗ no pic for post 1
2 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:me25ptvza6pvbhldxmslfski/bafkreibp66miin4cb4zgwsuzbozpqfvgsxykheypddcmm6y3xf3jdrlc24@jpeg
    found cat label: -1
    found bad label: 36 [917, 'comic book']
    passed cat test False
    ✓✗ post 2 was not a cat pic
3 ✗ you already follow sinnerbunny.bsky.social.
4 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:2gvkp6lxy6i7dugkza6q6tvh/bafkreiapf4hbaraoswd7mlplo642n2ftp4ddqov4aryejcypnjgxfcnfn4@jpeg
    found cat label: -1
    found bad label: -1 []
    passed cat test False
    ✓✗ post 4 was not a cat pic
5 ✓ https://cdn.bsky.app/img/feed_fullsize/plain/did:plc:mda5p64chnynpei53huhdv4g/bafkreiea4he5ch6mtpxanhgnz7rsy34a6d6nujtdnutm2rahyfww2sjpma@jpeg
    found cat label: -1
    found bad label: -1 []
    passed cat test False
    ✓✗ post 5 was not a cat pic
6 ✗ you already follow gazamcnulty.bsky.social.
7 ✗ you

True